## Import required lib

In [3]:

pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install SimpleITK

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install nibabel

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ToTensor, Normalize
import nibabel as nib
from scipy.ndimage import zoom
import torch.nn as nn
from torchvision.models.video import r3d_18
import pandas as pd
import SimpleITK as sitk
import pickle

In [7]:
final_df = pd.read_csv('/kaggle/input/output/final_output.csv')
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2113 entries, 0 to 2112
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Subject        2113 non-null   object
 1   Age            2113 non-null   int64 
 2   Modality       2113 non-null   object
 3   Group          2113 non-null   object
 4   Image Data ID  2113 non-null   int64 
 5   path           2113 non-null   object
 6   filename       2113 non-null   object
dtypes: int64(2), object(5)
memory usage: 115.7+ KB


In [8]:
mri_df = final_df[final_df['Modality'] == 'MRI']
mri_paths = (mri_df['path'] +'/'+mri_df['filename']).to_numpy()
mri_labels = mri_df['Group'].to_numpy()

pet_df = final_df[final_df['Modality'] == 'PET']
pet_paths = (pet_df['path'] +'/'+pet_df['filename']).to_numpy()
pet_labels = pet_df['Group'].to_numpy()

mri_train_image_paths, mri_test_image_paths, mri_train_labels, mri_test_labels = train_test_split(
    mri_paths, mri_labels, test_size=0.3, random_state=42)


#PET
pet_train_image_paths, pet_test_image_paths, pet_train_labels, pet_test_labels = train_test_split(
    pet_paths, pet_labels, test_size=0.3, random_state=42)

## Load the excel file with paths

In [9]:
with open('/kaggle/input/pickle-files/mri_train_array.pkl', 'rb') as f:
    mri_train_images = pickle.load(f)

with open('/kaggle/input/pickle-files/mri_test_array.pkl', 'rb') as f:
    mri_test_images = pickle.load(f)
    
with open('/kaggle/input/pickle-files/pet_train_array.pkl', 'rb') as f:
    pet_train_images = pickle.load(f)

with open('/kaggle/input/pickle-files/pet_test_array.pkl', 'rb') as f:
    pet_test_images = pickle.load(f)

print(mri_train_images.shape)
print(mri_test_images.shape)

print(pet_train_images.shape)
print(pet_test_images.shape)


(510, 60, 128, 128)
(219, 60, 128, 128)
(968, 60, 128, 128)
(416, 60, 128, 128)


In [10]:
def count_classes(train_labels):
    unique_labels, counts = np.unique(train_labels, return_counts=True)
    for label, count in zip(unique_labels, counts):
        print(f"Number of images in class {label}: {count}")

In [11]:
mri_unique_labels, mri_counts = np.unique(mri_train_labels, return_counts=True)
for label, count in zip(mri_unique_labels, mri_counts):
    print(f"Number of images in class {label}: {count}")

Number of images in class AD: 67
Number of images in class CN: 149
Number of images in class MCI: 294


In [12]:
pet_unique_labels, pet_counts = np.unique(pet_train_labels, return_counts=True)
for label, count in zip(pet_unique_labels, pet_counts):
    print(f"Number of images in class {label}: {count}")

Number of images in class AD: 116
Number of images in class CN: 298
Number of images in class MCI: 554


In [13]:
# Number of images to extract for each label
num_images_per_label = {"AD": 67, "CN": 149, "MCI": 294}

# Extract images for each label
extracted_images = []
extracted_labels = []
for label in num_images_per_label:
    # Get indices of images with current label
    label_indices = np.where(pet_train_labels == label)[0]
    
    # Select random subset of images with current label
    selected_indices = np.random.choice(label_indices, size=num_images_per_label[label], replace=False)
    selected_images = pet_train_images[selected_indices]
    selected_labels = pet_train_labels[selected_indices]
    
    # Add selected images to extracted_images list
    extracted_images.append(selected_images)
    extracted_labels.append(selected_labels)
    

# Combine extracted images for all labels
pet_extracted_images = np.concatenate(extracted_images)
pet_extracted_labels = np.concatenate(extracted_labels)


In [14]:
pet_unique_labels, pet_counts = np.unique(pet_extracted_labels, return_counts=True)
for label, count in zip(pet_unique_labels, pet_counts):
    print(f"Number of images in class {label}: {count}")

Number of images in class AD: 67
Number of images in class CN: 149
Number of images in class MCI: 294


In [15]:
unique_labels, numerical_labels = np.unique(mri_train_labels, return_inverse=True)
sorted_indices = np.argsort(numerical_labels)
mri_sorted_images = mri_train_images[sorted_indices]
mri_sorted_labels = mri_train_labels[sorted_indices]

In [16]:
print(mri_sorted_images.shape)
print(mri_sorted_labels.shape)
print(mri_sorted_labels)
print(pet_extracted_images.shape)
print(pet_extracted_labels.shape)
print(pet_extracted_labels)

(510, 60, 128, 128)
(510,)
['AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD'
 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD'
 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD'
 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD'
 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'AD' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'
 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN' 'CN'

In [17]:
classes = np.array(['MCI', 'CN', 'AD'])
# Create a dictionary to map class names to their corresponding index values
class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}
# Use a list comprehension to convert the class names to index values
mri_labels_idx = np.array([class_to_idx[label] for label in mri_sorted_labels])
# Use the index values to create the one-hot encoded labels
mri_lb_oh = np.eye(len(classes))[mri_labels_idx]
mri_train_data_model = mri_sorted_images[:, np.newaxis, :,:,:]


pet_labels_idx = np.array([class_to_idx[label] for label in pet_extracted_labels])
pet_lb_oh = np.eye(len(classes))[pet_labels_idx]
pet_train_data_model = pet_extracted_images[:, np.newaxis, :,:,:]

In [18]:
print(mri_train_data_model.shape)
print(mri_lb_oh.shape)
print(pet_train_data_model.shape)
print(pet_lb_oh.shape)


(510, 1, 60, 128, 128)
(510, 3)
(510, 1, 60, 128, 128)
(510, 3)


## RESNET model implementation

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, ZeroPadding3D,\
     Flatten, BatchNormalization, AveragePooling3D, Dense, Activation, Add  , Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import optimizers

In [20]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(1, 1,1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # third block activation used after adding the input
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  # x = Activation(activations.relu)(x)

  # add the input 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv3D(f1, kernel_size=(1,1, 1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  # second block
  x = Conv3D(f1, kernel_size=(3,3, 3), strides=(1,1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)

  #third block
  x = Conv3D(f2, kernel_size=(1,1, 1), strides=(1,1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv3D(f2, kernel_size=(1, 1,1), strides=(s,s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation(activations.relu)(x)

  return x

In [21]:
def resnet50mri():

  input_im = Input(shape=(1,60, 128, 128,)) # cifar 10 images size,
  x = ZeroPadding3D(padding=(1,1, 1))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv3D(64, kernel_size=(3, 3,3), strides=(1,1, 1))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling3D((1,1, 1), strides=(1, 1,1))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))

  # 3rd stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 4th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 5th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # ends with average pooling and dense connection

  x = AveragePooling3D((2,2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(3, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [22]:
def resnet50pet():

  input_im = Input(shape=(1,60, 128, 128,)) # cifar 10 images size,
  x = ZeroPadding3D(padding=(1,1, 1))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = Conv3D(64, kernel_size=(3, 3,3), strides=(1,1, 1))(x)
  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)
  x = MaxPooling3D((1,1, 1), strides=(1, 1,1))(x)

  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_conv(x, s=1, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))
  x = res_identity(x, filters=(16, 64))

  # 3rd stage

  x = res_conv(x, s=2, filters=(64, 128))
  x = res_identity(x, filters=(64, 128))
  x = res_identity(x, filters=(64, 128))
  x = res_identity(x, filters=(64, 128))

  # 4th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 5th stage

  x = res_conv(x, s=2, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # ends with average pooling and dense connection

  x = AveragePooling3D((2,2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(3, activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model

In [23]:
base_model_MRI =resnet50mri()

fc_layer = tf.keras.layers.Dense(512, activation='relu')(base_model_MRI.layers[-2].output)
fc_layer = tf.keras.layers.Dropout(0.8)(fc_layer)
fc_layer = Dense(16)(fc_layer)
fc_layer = Model(inputs=base_model_MRI.input, outputs=fc_layer)


base_model_PET = resnet50pet()

fc_layer1 = tf.keras.layers.Dense(512, activation='relu')(base_model_PET.layers[-2].output)
fc_layer1 = tf.keras.layers.Dropout(0.8)(fc_layer1)
fc_layer1 = Dense(16)(fc_layer1)
fc_layer1 = Model(inputs=base_model_PET.input, outputs=fc_layer1)

z = Concatenate()([fc_layer.output, fc_layer1.output])
# z = Dense(16, activation="relu")(combined)
output_layer = tf.keras.layers.Dense(3, activation='sigmoid')(z)
model = tf.keras.models.Model(inputs=[base_model_MRI.input,base_model_PET.input], outputs=output_layer)

In [24]:
# model.summary()

In [25]:
mri_data = mri_train_data_model
pet_data = pet_train_data_model

labels = mri_lb_oh

In [26]:
print(mri_train_data_model.shape)
print(pet_train_data_model.shape)
print(labels[:15])

(510, 1, 60, 128, 128)
(510, 1, 60, 128, 128)
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [27]:
model.compile('adam','categorical_crossentropy',metrics=['accuracy'])

In [28]:
num_samples = len(labels)
shuffle_indices = np.random.permutation(num_samples)

# shuffle images and labels arrays using the shuffled indices
shuffled_mri_data = mri_data[shuffle_indices]
shuffled_pet_data = pet_data[shuffle_indices]
shuffled_labels = labels[shuffle_indices]

In [29]:
print(shuffled_mri_data.shape)
print(shuffled_pet_data.shape)
print(shuffled_labels[:15])

(510, 1, 60, 128, 128)
(510, 1, 60, 128, 128)
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [30]:
model.fit([shuffled_mri_data,shuffled_pet_data],shuffled_labels,batch_size=16,epochs=50,verbose=1)

Epoch 1/50
32/32 [==============================] - 432s 12s/step - loss: 22.5321 - accuracy: 0.4157
Epoch 2/50
32/32 [==============================] - 383s 12s/step - loss: 12.6106 - accuracy: 0.5000
Epoch 3/50
32/32 [==============================] - 366s 11s/step - loss: 10.8142 - accuracy: 0.4412
Epoch 4/50
32/32 [==============================] - 365s 11s/step - loss: 9.8204 - accuracy: 0.5137
Epoch 5/50
32/32 [==============================] - 374s 12s/step - loss: 8.7415 - accuracy: 0.5529
Epoch 6/50
32/32 [==============================] - 371s 12s/step - loss: 8.2657 - accuracy: 0.5490
Epoch 7/50
32/32 [==============================] - 368s 11s/step - loss: 8.0916 - accuracy: 0.5804
Epoch 8/50
32/32 [==============================] - 363s 11s/step - loss: 7.8864 - accuracy: 0.5706
Epoch 9/50
32/32 [==============================] - 367s 11s/step - loss: 7.7690 - accuracy: 0.5902
Epoch 10/50
32/32 [==============================] - 370s 12s/step - loss: 7.7123 - accuracy: 0.6

In [35]:
from keras.models import load_model
model.save('mri_pet.h5')

In [32]:
mri_test_images = np.array(mri_test_images)
print(mri_test_images.shape)
print(mri_test_labels)

# Use a list comprehension to convert the class names to index values
mri_test_labels_idx = np.array([class_to_idx[label] for label in mri_test_labels])

# Use the index values to create the one-hot encoded labels
one_hot_test_labels = np.eye(len(classes))[mri_test_labels_idx]
mri_test_data_model = mri_test_images[:, np.newaxis, :,:,:]
print(mri_test_data_model.shape)
print(mri_test_labels_idx)


(219, 60, 128, 128)
['MCI' 'MCI' 'CN' 'MCI' 'AD' 'MCI' 'CN' 'AD' 'CN' 'MCI' 'MCI' 'AD' 'CN'
 'CN' 'CN' 'AD' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI' 'CN' 'CN' 'AD' 'CN' 'MCI'
 'CN' 'MCI' 'AD' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'CN' 'CN'
 'CN' 'MCI' 'MCI' 'CN' 'MCI' 'CN' 'CN' 'MCI' 'CN' 'AD' 'MCI' 'CN' 'CN'
 'AD' 'AD' 'MCI' 'MCI' 'CN' 'MCI' 'MCI' 'CN' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI'
 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'MCI' 'CN' 'CN' 'MCI' 'MCI' 'MCI' 'MCI'
 'MCI' 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'AD' 'MCI' 'CN' 'MCI' 'MCI' 'MCI'
 'MCI' 'AD' 'CN' 'CN' 'MCI' 'AD' 'MCI' 'MCI' 'AD' 'MCI' 'AD' 'CN' 'MCI'
 'CN' 'CN' 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI' 'MCI' 'AD'
 'MCI' 'MCI' 'MCI' 'CN' 'CN' 'MCI' 'CN' 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'MCI'
 'MCI' 'AD' 'AD' 'AD' 'AD' 'MCI' 'MCI' 'MCI' 'CN' 'CN' 'CN' 'MCI' 'CN'
 'MCI' 'MCI' 'MCI' 'AD' 'MCI' 'CN' 'MCI' 'MCI' 'MCI' 'MCI' 'AD' 'CN' 'MCI'
 'CN' 'CN' 'MCI' 'MCI' 'CN' 'CN' 'MCI' 'MCI' 'MCI' 'CN' 'MCI' 'MCI' 'CN'
 'MCI' 'MCI' 'AD' 'MCI' 'CN' 'CN' '

In [33]:
y_pred_probs = model.predict([mri_test_data_model,mri_test_data_model])  # Predicted probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Compute the confusion matrix
confusion_mat = confusion_matrix(mri_test_labels_idx, y_pred)

# Compute evaluation metrics
accuracy = accuracy_score(mri_test_labels_idx, y_pred)
precision = precision_score(mri_test_labels_idx, y_pred, average='weighted')
recall = recall_score(mri_test_labels_idx, y_pred, average='weighted')
f1score = f1_score(mri_test_labels_idx, y_pred, average='weighted')

print("Confusion Matrix:\n", confusion_mat)
print("Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1score))

7/7 [==============================] - 47s 6s/step
Confusion Matrix:
 [[118   5   1]
 [  6  56   3]
 [ 10   0  20]]
Accuracy: 0.8858
Precision: 0.8852
Recall: 0.8858
F1-score: 0.8832


In [34]:
print(mri_test_labels_idx)

[0 0 1 0 2 0 1 2 1 0 0 2 1 1 1 2 0 0 0 0 0 1 1 2 1 0 1 0 2 0 0 0 0 0 0 1 0
 1 1 1 0 0 1 0 1 1 0 1 2 0 1 1 2 2 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0
 0 0 0 0 0 0 0 1 0 2 0 1 0 0 0 0 2 1 1 0 2 0 0 2 0 2 1 0 1 1 0 0 0 1 0 0 0
 0 0 0 2 0 0 0 1 1 0 1 0 0 0 1 0 0 0 2 2 2 2 0 0 0 1 1 1 0 1 0 0 0 2 0 1 0
 0 0 0 2 1 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 2 0 1 1 0 0 1 1 2 0 0 0 1 1 0 2
 2 0 1 0 0 1 0 2 0 2 1 0 0 1 1 0 1 0 0 2 0 0 2 1 1 0 2 1 0 1 0 0 0 0]


In [ ]:
#print(test_data_model)